In [7]:
import cv2
import numpy as np
from PIL import Image
from IPython.display import display, clear_output

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # adjust width
cap.set(4, 480)  # adjust height

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Define color ranges in HSV for ripeness and stale detection
colors = {
    'Unripe (Green)': ([35, 40, 40], [85, 255, 255]),  # Green
    'Ripe (Yellow)': ([20, 100, 100], [40, 255, 255]),  # Adjusted range to include light yellow
    'Overripe (Brown)': ([10, 100, 20], [20, 255, 200]),  # Brownish/Orange
    'Stale (Black)': ([0, 0, 0], [180, 255, 50]),  # Black
}

# Define the region of interest (ROI) for the hand/object (adjust based on your setup)
roi_x_start, roi_y_start = 160, 240  # Top-left corner of the ROI
roi_width, roi_height = 320, 240     # Width and height of the ROI

try:
    while True:
        success, img = cap.read()
        if not success:
            print("Error: Failed to capture image.")
            break

        # Define the ROI where the hand/object is expected to be
        roi = img[roi_y_start:roi_y_start + roi_height, roi_x_start:roi_x_start + roi_width]

        # Convert the ROI to HSV color space
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

        # Total pixels in the ROI
        total_pixels = roi.shape[0] * roi.shape[1]

        dominant_color = "Unknown"
        max_color_percent = 0
        largest_contour = None
        bounding_box = None
        bounding_box_text = ""

        for color_name, (lower, upper) in colors.items():
            lower = np.array(lower)
            upper = np.array(upper)

            # Create a mask for the color within the ROI
            mask = cv2.inRange(hsv, lower, upper)

            # Calculate the percentage of the ROI that is this color
            color_pixels = cv2.countNonZero(mask)
            color_percent = (color_pixels / total_pixels) * 100

            if color_percent > max_color_percent:
                max_color_percent = color_percent
                dominant_color = color_name

            # Find contours in the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            # Identify the largest contour
            if contours:
                largest_contour = max(contours, key=cv2.contourArea)
                if cv2.contourArea(largest_contour) > 500:  # Filter out small areas
                    x, y, w, h = cv2.boundingRect(largest_contour)
                    bounding_box = (x, y, w, h)
                    bounding_box_text = f'{color_name}: {color_percent:.2f}%'

        # Draw the largest bounding box and its annotation if found
        if bounding_box:
            x, y, w, h = bounding_box
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color rectangle
            cv2.putText(roi, bounding_box_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)  # Red text on top of the box

        # Determine edibility based on color detection
        if dominant_color == 'Stale (Black)':
            edibility = "Not Edible (Stale)"
        elif dominant_color == 'Ripe (Yellow)':
            edibility = "Edible"
        else:
            edibility = "Not Edible"

        # Display edibility status and dominant color at the top in smaller white text
        cv2.putText(roi, f'Status: {edibility}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)  # Smaller white text
        cv2.putText(roi, f'Dominant Color: {dominant_color}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)  # Smaller white text

        # Convert the image to RGB (from BGR) for display in Jupyter
        img_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        clear_output(wait=True)
        display(Image.fromarray(img_rgb))

except KeyboardInterrupt:
    print("Stopped by the user")

finally:
    # Release the camera
    cap.release()


Stopped by the user


In [31]:
from PIL import Image
from IPython.display import display, HTML

# Title of the display
title = "Classification of Objects Trial - (Banana) Based on Color"

# Paths to the images
image_paths = {
    'Ripe (Yellow)': r'C:\Users\nirmi\yellow.png',
    'Unripe (Green)': r'C:\Users\nirmi\green.png',
    'Stale (Black)': r'C:\Users\nirmi\black.png'
}

# Start the HTML block with the title
html_content = f"<h2 style='text-align:center;'>{title}</h2><div style='display:flex; justify-content:center;'>"

# Loop through each image and add it to the HTML content
for label, path in image_paths.items():
    # Load the image
    image = Image.open(path)
    
    # Save the resized image to a temporary file
    temp_path = f"temp_{label.replace(' ', '_').replace('(', '').replace(')', '').lower()}.png"
    image = image.resize((200, 200))  # Resize to fit in one row
    image.save(temp_path)
    
    # Add the image and its label to the HTML content
    html_content += f"""
    <div style="text-align:center; margin: 10px;">
        <img src='{temp_path}' style="width:200px; height:200px;">
        <div style='color:black;'>{label}</div>
    </div>
    """

# Close the HTML block
html_content += "</div>"

# Display the title and images with labels
display(HTML(html_content))
